In [3]:
import pandas as pd
from zone_recovery.zone import *

In [3]:
test = Zone(1,2,3,4)

In [49]:
df  = pd.read_csv('processed_data/BNBUSTD_2018_Processed.csv')

In [52]:
df[0]

KeyError: 0

In [55]:
for i,r in df[5:10].iterrows():
    print(r.Price + 5)

11.1162
11.117999999999999
11.1196
11.1172
11.1065


In [6]:
for i,row in df[0:10].iterrows():
    print(i)

0
1
2
3
4
5
6
7
8
9


In [12]:
import numpy as numpy


class Zone:
    def __init__(self, entry_price, zone_width, distance):
        "entry_price"
        self.up = entry_price
        self.down = entry_price - zone_width
        self.above_target = entry_price + distance
        self.below_target = entry_price - zone_width - distance

    def calculateProfit(self, t, df, money):
        state = 'Sell'
        first_money = money
        buy_cost = 0
        sell_cost = 0

        buy_unit = 0
        sell_unit = 0
        alpha = 1

        temp_money = 0

        for i, row in df.iterrows():

            if row.Price > buy_cost and state == 'Buy':
                buy_cost += 50 * alpha
                alpha *= 2
                money -= buy_cost
                buy_unit += buy_cost / row.Price
                state = 'Sell'
                print('Buy', buy_cost)

            if row.Price > sell_cost and state == 'Sell':
                sell_cost += 50 * alpha
                temp_money += sell_cost

                alpha *= 2
                money -= sell_cost
                sell_unit += sell_cost / row.Price
                state = 'Buy'
                print('Sell', sell_cost)

            if row.Price > self.above_target and state == 'Sell':
                buy_cost = 0
                money += buy_unit * row.Price

                state = 'Buy'
                print('Close Buy', buy_unit * row.Price)
                buy_unit = 0
                alpha = 1

            if row.Price < self.below_target and state == 'Buy':
                sell_cost = 0
                debt = sell_unit * row.Price
                money += temp_money - debt
                sell_unit = 0
                state = 'Sell'
                print('Close Sell', temp_money - debt)
                alpha = 1

        return money - first_money

    def monitorZone(self):
        return self.up, self.down, self.above_target, self.below_target


In [40]:
first_price = df.iloc[0].Price

In [41]:
first_price

6.1222

In [14]:
z = Zone(1,2,3)

In [15]:
z.monitorZone()

(1, -1, 4, -4)

In [5]:
def makeDecision(prev_price, current_price, zone: Zone , state : str):

    if (prev_price > zone.up and current_price < zone.up or current_price == zone.up) and state == 'w8 4 Buy':
        return 'Open Buy', 'w8 4 Sell'

    if (prev_price < zone.down and current_price > zone.down or current_price == zone.down) and state == 'w8 4 Sell':
        return 'Open Sell', 'w8 4 Buy'

    if (prev_price < zone.above_target and current_price > zone.above_target or current_price == zone.above_target) and state == 'w8 4 Sell':
        return 'Close Buy', 'w8 4 Buy'

    if (prev_price > zone.below_target and current_price < zone.below_target or current_price == zone.below_target) and state == 'w8 4 Buy':
        return 'Close Sell', 'w8 4 Sell'

    return 'Steady', state



In [43]:
def simulation(df,limit=4,alpha=2,money=1000):


    state = 'w8 4 Sell'
    action = 'Open Buy'
    first_money = money
    temp_money = 0
    buy_cost =  1 
    sell_cost = 0
    
    sell_unit = 0
    
    first_price = df.iloc[0].Price
    prev_price = first_price
    buy_unit = buy_cost / first_price

    money -= buy_cost
    print(action,first_price,money,1,0)

    zone = Zone(first_price,first_price * 0.01, first_price * 0.04)

    buy_hold = 1
    sell_hold = 0
    
    for i,row in df[1:].iterrows():
        
        current_price = row.Price
        action, state = makeDecision(prev_price, current_price,zone,state)

        if alpha > 8:

            buy_cost = 0
            money += buy_unit * row.Price
            buy_unit = 0
          

            sell_cost = 0
            debt = sell_unit * row.Price
            money += temp_money - debt
            sell_unit = 0
            buy_hold,sell_hold = 0,0
            alpha = 1
            action = 'Cut Loss'
    
        
        if action == 'Open Buy':
            buy_cost += 1 * alpha
            buy_hold+= alpha
            alpha *= 2
            money -= buy_cost
            buy_unit += buy_cost / row.Price
            
            # state = 'Sell'
            # print('Open Buy', buy_cost, row.Price,money)
            

        if action == 'Open Sell':
            sell_cost += 1 * alpha
            sell_hold+=  alpha
            temp_money += sell_cost
            alpha *= 2
            money -= sell_cost
            sell_unit += sell_cost / row.Price
         
            # state = 'Buy'
            # print('Open Sell', sell_cost,money)


        if action == 'Close Buy':
            

            buy_cost = 0
            money += buy_unit * row.Price

            # state = 'Buy'
            # print('Close Buy', buy_unit * row.Price,money)
            buy_hold = 0
            buy_unit = 0
            alpha = 1
            

        if action == 'Close Sell':
            
            
            sell_cost = 0
            debt = sell_unit * row.Price
            # print(temp_money,debt)
            money += 2*temp_money - debt
            sell_unit = 0
            sell_hold = 0
            # state = 'Sell'
            # print('Close Sell', temp_money - debt,money)
            alpha = 1

        if sell_unit == 0 and buy_unit == 0:
            zone = Zone(current_price,current_price* 0.01, current_price* 0.04)



        if(action != 'Steady'):
            print(action,current_price,money,buy_hold,sell_hold)

        prev_price = current_price
    print(money-first_money)





In [26]:
test_zone = Zone(10,5,5)

In [27]:
test_zone.monitorZone()

(10, 5, 15, 0)

In [47]:
makeDecision(-2,-1,test_zone,'w8 4 Buy')

('Steady', 'w8 4 Buy')

In [50]:
simulation(df)

Open Buy 8.3959 999 1 0
Open Sell 8.3147 997 1 2
Open Buy 8.38 992 5 2
Open Sell 8.3125 982 5 10
Cut Loss 8.3017 987.958181334042 0 0
Open Buy 8.3244 986.958181334042 1 0
Close Buy 8.66 987.9984965519557 0 0
Open Buy 8.65 986.9984965519557 1 0
Open Sell 8.6039 984.9984965519557 1 2
Close Sell 8.2019 1011.0919425473763 1 0
Open Sell 8.5722 1010.0919425473763 1 1
Close Sell 8.1574 1039.1403315257016 1 0
Open Sell 8.58 1038.1403315257016 1 1
Open Buy 8.65 1035.1403315257016 3 1
Open Sell 8.6 1030.1403315257016 3 5
Close Sell 8.2101 1066.4101294603252 3 0
Open Sell 8.581 1065.4101294603252 3 1
Open Buy 8.65 1060.4101294603252 5 1
Open Sell 8.575 1055.4101294603252 5 5
Open Buy 8.6289 1042.4101294603252 13 5
Cut Loss 8.6291 1085.351534511815 0 0
Close Buy 11.7872 1085.351534511815 0 0
Open Buy 11.33 1084.351534511815 1 0
Open Sell 11.2914 1082.351534511815 1 2
Close Sell 10.74 1138.4492017629973 1 0
Open Sell 11.23 1137.4492017629973 1 1
Open Buy 10.2801 1134.4492017629973 3 1
Open Sell 11.